## Import and configuration variables

In [72]:
import csv
import re
import requests
import os

csv_arts = "../data/retracted_arts.csv"
csv_history = "../data/retracted_history.csv"

## Usefull functions

In [131]:
def write_list(l,file_path, header= True):
    f = open(file_path,"w+")
    initial_pos = 0
    
    #header
    if header:
        initial_pos = 1
        str_header = ''
        for k_header in l[0].keys():
            str_header = str_header + str(k_header) + ","
        f.write(str_header[:-1]+"\n")
        
    #content
    for l_index in range(initial_pos,len(l)):
        str_row = ''
        for k_att in l[l_index]:
            str_row = str_row + '"'+str(l[l_index][k_att]) +'"'+','
        f.write(str_row[:-1]+"\n")

## The main program  

The elaboratio steps to follow are: 
1. Get the COCI metadata for each retracted article in the csv
2. Write the results on a csv file 
3. Select the subset of elements from the list of all retracted items which have been found in COCI
4. Write the results of (3) on a csv file

In [74]:
#(1) Get the COCI metadata for each retracted article in the csv
items = []
with open(csv_history) as a_file:
    csv_reader = csv.reader(a_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        
        regex = r"(10\..*)$"

        test_str = row[4]

        matches = re.finditer(regex, test_str, re.MULTILINE)

        for matchNum, match in enumerate(matches, start=1):
            if match:
                doi = match.group()
                r = requests.get('https://opencitations.net/index/coci/api/v1/metadata/'+str(doi))
                items.append(r.json()[0])

In [75]:
#(2) Write the results on a csv file
write_list(items,"../data/retracted_history_(coci_metadata).csv")

In [136]:
# (3) Select the subset of elements from the list of all retracted items which have been found in COCI

#History articles in COCI (according to the results obtained in step 1)
history_dois = ["10.1080/014198700329114","10.1111/j.1466-8238.2009.00501.x","10.3171/2010.9.focus10200","10.1016/j.shpsa.2011.02.001","10.1080/17430437.2014.919258","10.1093/jis/etp023","10.3171/2012.6.focus12120","10.1093/jogss/ogw014","10.1177/0267323115589265","10.1007/1-4020-3001-0_4","10.2105/ajph.2015.302623","10.1186/1744-859x-13-11","10.1080/13825571003588429","10.2143/etl.75.4.504765","10.2143/rtpm.67.1.520","10.1260/136943306779369509","10.1017/ccol0521822424.012","10.1017/ccol0521650186.018"]
#Arts articles in COCI 
arts_dois = ["10.1016/j.jesp.2011.02.006","10.1111/j.1365-2702.2010.03368.x","10.1109/esiat.2010.5568967","10.1109/iccsit.2010.5564505","10.1007/s11059-013-0186-9","10.1353/cdr.2011.0017","10.1109/iccsit.2010.5564489","10.1109/edt.2010.5496628"]

subset_items = []
with open(csv_arts, 'r') as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        regex = r"(10\..*)$"
        matches = re.finditer(regex, row[4], re.MULTILINE)
        for matchNum, match in enumerate(matches, start=1):
            if match:
                doi = match.group()
                if doi.lower() in arts_dois:
                    rowdict = {}
                    for j in range(0,len(row)):
                        rowdict[j] = row[j]
                    subset_items.append(rowdict)

#order them according to the results
sorted_list = []
for doi in arts_dois:
    for item in subset_items:
        if doi.lower() in item[4].lower():
            sorted_list.append(item)

In [137]:
#(4) Write the results of (3) on a csv file
write_list(sorted_list,"../data/retracted_arts_(temp).csv", False)